This script (1) compiles published literature and (2) generates areas of interest for 3DEP downloads

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob

plt.rcParams.update({"pdf.fonttype":42})

# Gather all geologic maps for Ordovician and Silurian rocks

In [ ]:
o_s_polys = gpd.GeoDataFrame()

for state in glob.glob("./geology/*_geol_poly.shp"):
    state_geol = gpd.read_file(state)
    state_o_s = state_geol[(state_geol['ORIG_LABEL'].str.contains('S')) | (state_geol['ORIG_LABEL'].str.contains('O'))]
    o_s_polys = gpd.GeoDataFrame(pd.concat([o_s_polys, state_o_s], ignore_index=True))

In [ ]:
liths = pd.DataFrame()

for state in glob.glob("./geology/*_lith.csv"):
    state_liths = pd.read_csv(state)
    liths = pd.concat([liths, state_liths], ignore_index=True)


In [ ]:
units = pd.DataFrame()

for state in glob.glob("./geology/*_units.csv"):
    state_units = pd.read_csv(state)
    units = pd.concat([units, state_units], ignore_index=True)

descripts = pd.merge(units, liths, on="unit_link")

all_data = pd.merge(o_s_polys, descripts, left_on="UNIT_LINK", right_on="unit_link")
all_data = all_data.dissolve(by='ORIG_LABEL').reset_index()
# all_data.to_file("all_data.shp")

# Find large-scale trends in relief

## Make latitude bins

In [ ]:
from shapely.geometry import Polygon
import numpy as np

ymin=31
ymax=45
xmin=-88
xmax=-73
width = abs(xmin)-abs(xmax)
height = 0.125
rows = int(np.ceil((ymax-ymin) /  height))
cols = int(np.ceil((xmax-xmin) / width))
XleftOrigin = xmin
XrightOrigin = xmin + width
YtopOrigin = ymax
YbottomOrigin = ymax- height
polygons = []
for i in range(cols):
    Ytop = YtopOrigin
    Ybottom =YbottomOrigin
for j in range(rows):
    polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)])) 
    Ytop = Ytop - height
    Ybottom = Ybottom - height
XleftOrigin = XleftOrigin + width
XrightOrigin = XrightOrigin + width

grid = gpd.GeoDataFrame({'geometry':polygons}).set_crs("EPSG:4326")

# https://gis.stackexchange.com/questions/269243/creating-polygon-grid-using-geopandas

In [ ]:
grid['lat_max'] = [(x.exterior.coords)[2][1] for x in polygons]
grid['lat_min'] = [(x.exterior.coords)[0][1] for x in polygons]

In [ ]:
all_data_by_lat_bin = gpd.overlay(all_data, grid, how='intersection')

In [ ]:
tuscarora = all_data_by_lat_bin[all_data_by_lat_bin['unit_name'].str.contains("Tuscarora")].to_crs('EPSG:26917')


## Get relief raster and do some stats

In [ ]:
import rasterio
from rasterio import features
from shapely.geometry import box, shape

with rasterio.open("./topo/Central_App_relief_2500m_resample1.tif", masked=True) as relief:
    print(relief.crs)
    relief_meta = relief.profile
    relief_arr = relief.read(1)

### Get Tuscarora relief

In [ ]:
means_dict = {}
raw_values_dict = {}

for geom, idx in zip(tuscarora['geometry'], tuscarora.index):
    # I spent WAY too much time messing around with this part, you apparently can't just
    # give rasterize a polygon, it has to either be a MutliPolygon or a list of geometries
    # This code does the latter, and the Arctic Data Center tutorial's only works because their
    # example vector data is accidentally multipolygons, but our data has a polygon
    geom = shape(geom)
    geoms = [(geom, 1)]
    # Now this looks like the ADC example
    rasterized = features.rasterize(
                                    geoms,
                                    out_shape=relief_arr.shape, # Look like the source raster
                                    transform=relief_meta['transform'], # Have the geometry of the source raster
                                    all_touched=True # all pixels touched by geometries (as opposed to pixel centers)
                                    )
    # Theoretically instead of individually rasterizing each polygon type
    # You could rasterize the whole thing and instead of 0s and 1s you can
    # make the raster value the index and then do basic array-style analyses
    # Maybe I can offer treats to someone who writes that for me...
    r_index = np.where(rasterized == 1) # Make the mask
    raw_values_dict[idx] = relief_arr[r_index] # store all non-masked data (the compressed thing)
    means_dict[idx] = np.nanmean(relief_arr[r_index]) # calculate the mean of the data

In [ ]:
means_df = pd.DataFrame.from_dict(means_dict,
                                     orient='index',
                                     columns=['mean_relief'])

In [ ]:
tuscarora_relief = tuscarora.merge(means_df,
                                    left_index=True,
                                    right_index=True,
                                    how='inner')

### Get relief by province

In [ ]:
province_relief = {}
provinces = gpd.read_file("./cartography/physio.shp").to_crs('EPSG:4326').dissolve(by='PROVINCE').reset_index()
# province_list = list(provinces['PROVINCE'].unique())

province_list = [
 'APPALACHIAN PLATEAUS',
 'BLUE RIDGE',
#  'COASTAL PLAIN',
#  'PIEDMONT',
 'VALLEY AND RIDGE',
]

for province in province_list:
    subdict = {}
    prov_by_lat_bin = gpd.overlay(provinces[provinces['PROVINCE']==province], grid, how='intersection').to_crs('EPSG:26917')
    subdict['overlay'] = prov_by_lat_bin

    means_dict = {}
    # raw_values_dict = {}

    for geom, idx in zip(prov_by_lat_bin['geometry'], prov_by_lat_bin.index):
        # I spent WAY too much time messing around with this part, you apparently can't just
        # give rasterize a polygon, it has to either be a MutliPolygon or a list of geometries
        # This code does the latter, and the Arctic Data Center tutorial's only works because their
        # example vector data is accidentally multipolygons, but our data has a polygon
        geom = shape(geom)
        geoms = [(geom, 1)]
        # Now this looks like the ADC example
        rasterized = features.rasterize(
                                        geoms,
                                        out_shape=relief_arr.shape, # Look like the source raster
                                        transform=relief_meta['transform'], # Have the geometry of the source raster
                                        all_touched=True # all pixels touched by geometries (as opposed to pixel centers)
                                        )
        # Theoretically instead of individually rasterizing each polygon type
        # You could rasterize the whole thing and instead of 0s and 1s you can
        # make the raster value the index and then do basic array-style analyses
        # Maybe I can offer treats to someone who writes that for me...
        r_index = np.where(rasterized == 1) # Make the mask
        # r_index.filled(np.nan)
        # raw_values_dict[idx] = relief_arr[r_index] # store all non-masked data (the compressed thing)
        means_dict[idx] = np.nanmean(relief_arr[r_index]) # calculate the mean of the data

    means_df = pd.DataFrame.from_dict(means_dict,
                                        orient='index',
                                        columns=['mean_relief'])

    prov_relief = prov_by_lat_bin.merge(means_df,
                                        left_index=True,
                                        right_index=True,
                                        how='inner')
    subdict['relief'] = prov_relief.dropna()
    subdict['relief'] = prov_relief.loc[prov_relief['mean_relief'] < 1001, :]

    province_relief[province] = subdict
    

    

for province in province_list:
    subdict = {}
    prov_by_lat_bin = gpd.overlay(provinces[provinces['PROVINCE']==province], grid, how='intersection').to_crs('EPSG:26917')
    subdict['overlay'] = prov_by_lat_bin

    means_dict = {}
    # raw_values_dict = {}

    for geom, idx in zip(prov_by_lat_bin['geometry'], prov_by_lat_bin.index):
        # I spent WAY too much time messing around with this part, you apparently can't just
        # give rasterize a polygon, it has to either be a MutliPolygon or a list of geometries
        # This code does the latter, and the Arctic Data Center tutorial's only works because their
        # example vector data is accidentally multipolygons, but our data has a polygon
        geom = shape(geom)
        geoms = [(geom, 1)]
        # Now this looks like the ADC example
        rasterized = features.rasterize(
                                        geoms,
                                        out_shape=relief_arr.shape, # Look like the source raster
                                        transform=relief_meta['transform'], # Have the geometry of the source raster
                                        all_touched=True # all pixels touched by geometries (as opposed to pixel centers)
                                        )
        # Theoretically instead of individually rasterizing each polygon type
        # You could rasterize the whole thing and instead of 0s and 1s you can
        # make the raster value the index and then do basic array-style analyses
        # Maybe I can offer treats to someone who writes that for me...
        r_index = np.where(rasterized == 1) # Make the mask
        # r_index.filled(np.nan)
        # raw_values_dict[idx] = relief_arr[r_index] # store all non-masked data (the compressed thing)
        means_dict[idx] = np.nanmean(relief_arr[r_index]) # calculate the mean of the data

    means_df = pd.DataFrame.from_dict(means_dict,
                                        orient='index',
                                        columns=['mean_relief'])

    prov_relief = prov_by_lat_bin.merge(means_df,
                                        left_index=True,
                                        right_index=True,
                                        how='inner')
    subdict['relief'] = prov_relief.dropna()
    subdict['relief'] = prov_relief.loc[prov_relief['mean_relief'] < 1001, :]

    province_relief[province] = subdict
    

    

for province in province_list:
    subdict = {}
    prov_by_lat_bin = gpd.overlay(provinces[provinces['PROVINCE']==province], grid, how='intersection').to_crs('EPSG:26917')
    subdict['overlay'] = prov_by_lat_bin

    means_dict = {}
    # raw_values_dict = {}

    for geom, idx in zip(prov_by_lat_bin['geometry'], prov_by_lat_bin.index):
        # I spent WAY too much time messing around with this part, you apparently can't just
        # give rasterize a polygon, it has to either be a MutliPolygon or a list of geometries
        # This code does the latter, and the Arctic Data Center tutorial's only works because their
        # example vector data is accidentally multipolygons, but our data has a polygon
        geom = shape(geom)
        geoms = [(geom, 1)]
        # Now this looks like the ADC example
        rasterized = features.rasterize(
                                        geoms,
                                        out_shape=relief_arr.shape, # Look like the source raster
                                        transform=relief_meta['transform'], # Have the geometry of the source raster
                                        all_touched=True # all pixels touched by geometries (as opposed to pixel centers)
                                        )
        # Theoretically instead of individually rasterizing each polygon type
        # You could rasterize the whole thing and instead of 0s and 1s you can
        # make the raster value the index and then do basic array-style analyses
        # Maybe I can offer treats to someone who writes that for me...
        r_index = np.where(rasterized == 1) # Make the mask
        # r_index.filled(np.nan)
        # raw_values_dict[idx] = relief_arr[r_index] # store all non-masked data (the compressed thing)
        # means_dict[idx] = np.nanmean(relief_arr[r_index]) # calculate the mean of the data
        if relief_arr[r_index].size:
            means_dict[idx] = np.nanmean(relief_arr[r_index]) # calculate the mean of the data

    means_df = pd.DataFrame.from_dict(means_dict,
                                        orient='index',
                                        columns=['mean_relief'])

    prov_relief = prov_by_lat_bin.merge(means_df,
                                        left_index=True,
                                        right_index=True,
                                        how='inner')
    subdict['relief'] = prov_relief.dropna()
    subdict['relief'] = prov_relief.loc[prov_relief['mean_relief'] < 1001, :]

    province_relief[province] = subdict
    

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(12,3), dpi=200)
# sns.scatterplot(x='lat_min', y='mean_relief', data=tuscarora_relief, hue='ORIG_LABEL')
for province in list(province_relief.keys()):
    sns.lineplot(x='lat_min', y='mean_relief', data=province_relief[province]['relief'], label=province)
sns.scatterplot(x='lat_min', y='mean_relief', data=tuscarora_relief, hue='ORIG_LABEL')
ax.set_ylabel('Mean relief (m)')
ax.set_xlim(36.5, 41.5)
plt.savefig("mean_relief_lat_bin_tuscarora.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(12,3), dpi=200)
# sns.scatterplot(x='lat_min', y='mean_relief', data=tuscarora_relief, hue='ORIG_LABEL')
for province in list(province_relief.keys()):
    sns.lineplot(x='lat_min', y='mean_relief', data=province_relief[province]['relief'], label=province)
sns.scatterplot(x='lat_min', y='mean_relief', data=tuscarora_relief, hue='ORIG_LABEL')
ax.set_ylabel('Mean relief (m)')
ax.set_xlim(36.5, 41.5)
ax.set_ylim(600,50)
plt.savefig("mean_relief_lat_bin_tuscarora_r.pdf")

# Let's do the (Paul) numbers

this is a marketplace joke

## 2019 potomac compilation

In [ ]:
pdr1 = pd.read_csv("./be10/portenga_dr1.csv")
pdr3 = pd.read_csv("./be10/portenga_dr3.csv")

In [ ]:
portenga_2019 = pd.merge(pdr1, pdr3, left_on="Sample Ida", right_on="Sample IDa", how="outer")
portenga_2019 = portenga_2019.dropna(how="all", axis=0).dropna(how="all", axis=1)
portenga_2019['Long (dd)'] = portenga_2019['Longitude (°W)'].values*-1
portenga_2019.loc[:,'published_in'] = 'Portegna et al 2019'

In [ ]:
portenga_2019.columns

## gsa today 2011

In [ ]:
p11_basin = pd.read_csv("./be10/portenga_comp_basins.csv")
p11_outcrop = pd.read_csv("./be10/portenga_comp_outcrops.csv")

In [ ]:
portenga_2011 = pd.concat([p11_basin, p11_outcrop]).dropna(how="all")
portenga_2011.loc[:,'published_in'] = 'Portegna et al 2011'
portenga_2011['Sample type']=portenga_2011['Sample type'].fillna("Basin")
pd.to_numeric(portenga_2011['Lat (dd)'], errors="coerce")
portenga_2011['lat_bins'] = pd.cut(portenga_2011['Lat (dd)'], np.arange(30.5,42.0,0.5))

In [ ]:
portenga_2011.columns

In [ ]:
# Map 2019 data (first) to the 2011 compilation (second)
column_dict = {'Sample Ida' : 'Sample ID',
'Latitude (°N)' : 'Lat (dd)',
# 'Longitude (°W)' : 'Long (dd)',
'Unnested 10Bei erosion rate  (m Myr-1)b':'Published erosion rate',
'Avg. basin slope (°)c':'Basin slope',
'Basin area (km2)':'Basin Area',
'Relief ' : 'Basin relief (m)',
'published_in':'published_in'
}

portenga_all = pd.concat([portenga_2011, portenga_2019.rename(columns=column_dict)], ignore_index=True)

In [ ]:
portenga_all.columns

In [ ]:
# portenga_all=portenga_all.apply(pd.to_numeric,
# #  errors="ignore"
#  )
portenga_all['Published erosion rate'] = pd.to_numeric(portenga_all['Published erosion rate'], errors="coerce")


## Turn the sites into a geodataframe

In [ ]:
portenga_gdf = gpd.GeoDataFrame(
    portenga_all, geometry=gpd.points_from_xy(portenga_all['Long (dd)'], portenga_all['Lat (dd)']), crs='epsg:4326')

portenga_gdf = portenga_gdf.dropna(subset='Citation')

# Join Portenga data to geospatial data

## Join to provinces map

In [ ]:
portenga_gdf = gpd.sjoin(portenga_gdf, provinces).drop(columns=['index_right'])
portenga_gdf['MAP'] = portenga_gdf['MAP'].astype('float')

# Make squares for 3DEP

This is so that I can get tiles for 3DEP lidar data for LSDTT

In [ ]:
points = portenga_gdf[portenga_gdf['PROVINCE']=='VALLEY AND RIDGE'].filter(['Sample ID', 'geometry'])

def points_to_squares(gdf):
    # Takes lat long points
    # makes utm boxes
    # sends them back to lat long but boxes
    gdf['EPSG']= (
        32700-(np.round((45+gdf.geometry.y)/90,0)*100).astype(int)
        +np.round((183+gdf.geometry.x)/6,0).astype(int)
        ).astype(str)

 
    gdf['square_geom'] = np.nan
    for code in gdf['EPSG'].unique():
        code_subset = gdf.loc[gdf['EPSG'] == code]
        square = code_subset.to_crs(f'EPSG:{code}').buffer(1000, cap_style=3).to_crs("EPSG:4326")
        gdf.loc[gdf['EPSG'] == code, 'square_geom'] = square.geometry
        # list_4326.append([square.geometry])

    gdf_squares = gpd.GeoDataFrame(
        gdf.drop('geometry', axis=1).copy(), geometry=gdf.square_geom, crs="EPSG:4326"
        ).drop('square_geom', axis=1)
    
    return gdf_squares

portenga_squares = points_to_squares(points)

## Join 10Be data to geology map

In [ ]:
joined = gpd.sjoin(
    portenga_gdf, all_data,
 how="left"

 ).drop('index_right',
#  'index_left'],
 axis=1)

## Visualize 10Be data with geology and morphometry attributes

In [ ]:
joined['lat_stat'] = pd.cut(joined.geometry.y, [0,39,49])

I report these numbers in the text

In [ ]:
joined[joined['PROVINCE']=='VALLEY AND RIDGE']['Citation'].value_counts()

### Get morph attributes from publishd and from LSDTT output

In [ ]:
vr_basins = joined.loc[joined['PROVINCE']=='VALLEY AND RIDGE']

In [ ]:
vr_ridgelines = pd.read_csv('output_data/portenga_ridgelines_Cht.csv')
# vr_ridgelines

In [ ]:
# Find sites that are super close together and make a site average

numerical_cols = vr_ridgelines.select_dtypes(include=['number']).columns.tolist()

grouped_numerical = vr_ridgelines.groupby('Location')[numerical_cols].mean().reset_index()
grouped_numerical

vr_ridgelines_0 = pd.merge(grouped_numerical, vr_ridgelines.select_dtypes(exclude=['number']).groupby('Location').first().reset_index(),how='left')
vr_ridgelines= vr_ridgelines_0


In [ ]:
vr_ridgelines = vr_ridgelines.loc[vr_ridgelines['PROVINCE']=='VALLEY AND RIDGE']

In [ ]:
vr_basins.loc[:,'sample_cat'] = 'basin'
vr_ridgelines.loc[:,'sample_cat'] = 'ridgelines'

In [ ]:
merged_for_boxplots = pd.concat([vr_basins, vr_ridgelines])
merged_for_boxplots['lat_bins'] = pd.cut(merged_for_boxplots['Lat (dd)'], np.arange(37.5,42.0,0.5))

fig, ax=plt.subplots(figsize=(1,4),dpi=300)
sns.boxplot(data=merged_for_boxplots, y='lat_bins', x='CRONUS', hue='sample_cat', ax=ax, legend=False)
plt.yticks(rotation=45)
ax.invert_yaxis()
plt.savefig("portenga_vs_lat_all.pdf")

limit basins to 5 km

In [ ]:
vr_basins = vr_basins[vr_basins['Basin Area']<5]

In [ ]:
switch_lat = 39

vr_basins['lat_stat'] = pd.cut(vr_basins.geometry.y, [0,switch_lat,49])

vr_ridgelines['lat_stat'] = pd.cut(vr_ridgelines['Lat (dd)'], [0,switch_lat,49])

Make clean versions for DR

In [ ]:
vr_basins.to_csv('delvecchio_DR_basins.csv')

In [ ]:
pd.merge(vr_ridgelines, portenga_all, how='left', on='Sample ID').dropna(axis=1, how='all').to_csv('delvecchio_DR_ridgelines.csv')

In [ ]:
from matplotlib.ticker import FuncFormatter

from matplotlib.ticker import ScalarFormatter


fig, ax = plt.subplots(2,1, 
sharey=True,
figsize = (
  # 4.83,
  2.33,
  4), dpi=200)
sns.scatterplot(data=vr_ridgelines, 
                x=vr_ridgelines['Cht']*-1,
                 y='CRONUS', 
                #  y=pd.cut(vr_ridgelines['CRONUS'], e_bins),
                 hue='lat_stat', ax=ax[0], legend=False)
sns.scatterplot(data=vr_basins, x='Basin slope', 
                y='CRONUS',
                # y=pd.cut(vr_ridgelines['CRONUS'], e_bins),
                  hue='lat_stat', ax=ax[1], legend=False)
# ax[0].set_xlim(0.0025,0.03)
# ax[0].set_xscale('log')
ax[0].set_xticks([0.001, 0.01,0.02])

ax[0].set_xscale('log')



# ax[0].invert_xaxis()
ax[0].set_xlabel('-Cht')


# labels = [item.get_text() for item in ax[0].get_xticklabels()]
# labels = [f'{float(label):.2f}' for label in labels] # Format to 2 decimal places
# ax[0].set_xticklabels(labels)


fig.tight_layout()

ax[0].xaxis.set_major_formatter(ScalarFormatter())
ax[0].xaxis.get_major_formatter().set_scientific(False)
ax[0].xaxis.get_major_formatter().set_useOffset(False)

plt.savefig('figure_outputs/morph_erates.pdf')